Resources

https://notebooks.databricks.com/demos/llm-fine-tuning/index.html#

https://notebooks.databricks.com/demos/llm-rag-chatbot/index.html#

https://www.comet.com/site/blog/implementing-agents-in-langchain/

https://cohere.com/llmu

https://python.langchain.com/v0.2/docs/introduction/

https://huggingface.co/datasets

https://www.llamaindex.ai/

https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/

In [0]:
from pyspark.sql import Row


# Step 3: Define the Pokémon Data
pokemon_data = [
    {"name": "Pikachu", "power": "Electric"},
    {"name": "Charizard", "power": "Fire/Flying"},
    {"name": "Bulbasaur", "power": "Grass/Poison"},
    {"name": "Squirtle", "power": "Water"},
    {"name": "Jigglypuff", "power": "Fairy"},
    # Add more Pokémon characters and their powers
]

# Step 4: Convert the Data to a DataFrame
pokemon_df = spark.createDataFrame([Row(**i) for i in pokemon_data])

# Step 5: Show the DataFrame
pokemon_df.show()

+----------+------------+
|      name|       power|
+----------+------------+
|   Pikachu|    Electric|
| Charizard| Fire/Flying|
| Bulbasaur|Grass/Poison|
|  Squirtle|       Water|
|Jigglypuff|       Fairy|
+----------+------------+



In [0]:
pokemon_df.createOrReplaceTempView("pokemon")

In [0]:
df = spark.sql("""select * from pokemon""")
display(df)

name,power
Pikachu,Electric
Charizard,Fire/Flying
Bulbasaur,Grass/Poison
Squirtle,Water
Jigglypuff,Fairy


In [0]:
%pip install dbdemos

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import dbdemos; 

In [0]:
%pip install --quiet databricks-genai==1.0.8 mlflow==2.14.2

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%pip install --quiet databricks-sdk==0.18.0
%pip install --quiet databricks-genai==1.0.8
dbutils.library.restartPython()

Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pokemon_prompt = """
### Instruction
You are tasked as a Pokémon power categorization agent responsible for reading the descriptions of Pokémon characters and categorizing each into one of the predefined power categories. Please categorize each Pokémon into one of the following specific categories: 

Electric
Fire/Flying
Grass/Poison
Water
Fairy

Do not create or use any categories other than those explicitly listed above. Return only a single category as a response. If there is confusion between multiple categories, use your best judgment based on the description provided.

### Example Input
Pikachu is a small, yellow Pokémon that can generate electricity from the sacs in its cheeks. It uses electric attacks like Thunderbolt and can paralyze its opponents.

### Response
Electric

Based on the above categorize the following Pokémon: \n\n"""


In [0]:
%pip install databricks-api

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
spark.sql(f"""SELECT 
            ai_query("databricks-mixtral-8x7b-instruct", concat("{pokemon_prompt}", name)) AS mixtral_small_classification,
            name
        FROM pokemon 
        LIMIT 5""").display()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3260997823340358>:1
----> 1 spark.sql(f"""SELECT 
      2             ai_query("databricks-mixtral-8x7b-instruct", concat("{pokemon_prompt}", name1)) AS mixtral_small_classification,
      3             name1
      4         FROM pokemon 
      5         LIMIT 5""").display()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1119, in SparkSession.sql(self, sqlQuery, **kwargs)
   1117     sqlQuery = formatter.format(sqlQuery, **kwargs)
   1118 try:
-> 11

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Define the date range
start_date = datetime.strptime('2024-07-27', '%Y-%m-%d')
end_date = datetime.strptime('2024-07-31', '%Y-%m-%d')

# Generate a list of dates for the specified range
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Define the three country IDs
countries = ['IND', 'USA', 'NL']

# Define the number of unique IDs for sites and products
num_sites = 10
product_ids = [89, 91, 94, 98, 99]

# Generate data
data = []
for single_date in date_range:
    for country_id in countries:
        for site_id in range(1, num_sites + 1):
            for product_id in product_ids:
                volume_sold = round(random.uniform(100, 1000), 1)  # Random volume between 100 and 1000 with one decimal place
                if country_id == 'USA':
                    unit = 'gallons'
                    date_str = single_date.strftime('%m/%d/%Y')
                else:
                    unit = 'liters'
                    date_str = single_date.strftime('%d/%m/%Y')
                
                data.append([country_id, date_str, site_id, product_id, f"{volume_sold} {unit}"])

# Create DataFrame
columns = ['Country_ID', 'Date', 'Site_ID', 'Product_ID', 'Volume_Sold']
df = pd.DataFrame(data, columns=columns)

# Introduce 5% null values in 'Site_ID', 'Product_ID', and 'Volume_Sold'
num_nulls = int(0.05 * len(df))

# Randomly select indices for null values
null_indices = random.sample(range(len(df)), num_nulls * 3)

# Assign null values
for i, col in zip(null_indices[:num_nulls], ['Site_ID']*num_nulls):
    df.loc[i, col] = np.nan

for i, col in zip(null_indices[num_nulls:2*num_nulls], ['Product_ID']*num_nulls):
    df.loc[i, col] = np.nan

for i, col in zip(null_indices[2*num_nulls:], ['Volume_Sold']*num_nulls):
    df.loc[i, col] = np.nan

# Introduce 3% duplicate data
num_duplicates = int(0.03 * len(df))

# Randomly select indices for duplicates
duplicate_indices = random.sample(range(len(df)), num_duplicates)

# Create duplicates with different Volume_Sold
duplicates = df.loc[duplicate_indices].copy()
duplicates['Volume_Sold'] = [f"{round(random.uniform(100, 1000), 1)} {unit}" for _ in range(len(duplicates))]

# Concatenate the original DataFrame with the duplicates
df = pd.concat([df, duplicates], ignore_index=True)

# Display the first few rows of the dataset
display(df)

# Export to CSV
df.to_csv('/mnt/data/oil_consumption_dataset.csv', index=False)

Country_ID,Date,Site_ID,Product_ID,Volume_Sold
IND,27/07/2024,1.0,89.0,null
IND,27/07/2024,1.0,91.0,333.3 liters
IND,27/07/2024,1.0,94.0,804.5 liters
IND,27/07/2024,1.0,98.0,857.5 liters
IND,27/07/2024,1.0,99.0,906.8 liters
IND,27/07/2024,2.0,89.0,652.2 liters
IND,27/07/2024,2.0,91.0,666.2 liters
IND,27/07/2024,2.0,94.0,710.4 liters
IND,27/07/2024,2.0,98.0,225.5 liters
IND,27/07/2024,2.0,99.0,853.7 liters


---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
<command-84504009929124> in <cell line: 73>()
     71 
     72 # Export to CSV
---> 73 df.to_csv('/mnt/data/oil_consumption_dataset.csv', index=False)

/databricks/python/lib/python3.9/site-packages/pandas/core/generic.py in to_csv(self, path_or_buf, sep, na_rep, float_format, columns, header, index, index_label, mode, encoding, compression, quoting, quotechar, line_terminator, chunksize, date_format, doublequote, escapechar, decimal, errors, storage_options)
   3464         )
   3465 
-> 3466         return DataFrameRenderer(formatter).to_csv(
   3467             path_or_buf,
   3468             line_terminator=line_terminator,

/databricks/python/lib/python3.9/site-packages/pandas/io/formats/format.py in to_csv(self, path_or_buf, encoding, sep, columns, index_label, mode, compression, quoting, quotechar, line_terminator, chunksize, da

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Define the date range
start_date = datetime.strptime('2024-06-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-07-31', '%Y-%m-%d')

# Generate a list of dates for the specified range
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Define the three country IDs
countries = ['IND', 'USA', 'NL']

# Define the number of unique IDs for sites and products
num_sites = 10
product_ids = [89, 91, 94, 98, 99]

# Generate data
data = []
for single_date in date_range:
    for country_id in countries:
        for site_id in range(1, num_sites + 1):
            for product_id in product_ids:
                volume_sold = round(random.uniform(100, 1000), 1)  # Random volume between 100 and 1000 with one decimal place
                date_str = single_date.strftime('%Y-%m-%d')  # Use 'yyyy-MM-dd' format for all countries
                
                data.append([country_id, date_str, site_id, product_id, f"{volume_sold} liters"])

# Create DataFrame
columns = ['Country_ID', 'Date', 'Site_ID', 'Product_ID', 'Volume_Sold']
df = pd.DataFrame(data, columns=columns)

# Ensure no null values
# Since we generate all values programmatically, there should be no null values.
# But just to ensure, we will drop any potential nulls
df.dropna(inplace=True)

# Ensure no duplicate rows
df.drop_duplicates(inplace=True)

# Display the first few rows of the dataset
display(df)

# Export to CSV
df.to_csv('/mnt/data/oil_consumption_dataset.csv', index=False)


Country_ID,Date,Site_ID,Product_ID,Volume_Sold
IND,2024-06-01,1,89,293.6 liters
IND,2024-06-01,1,91,263.0 liters
IND,2024-06-01,1,94,463.8 liters
IND,2024-06-01,1,98,103.7 liters
IND,2024-06-01,1,99,909.5 liters
IND,2024-06-01,2,89,586.0 liters
IND,2024-06-01,2,91,417.7 liters
IND,2024-06-01,2,94,491.6 liters
IND,2024-06-01,2,98,597.3 liters
IND,2024-06-01,2,99,757.2 liters


---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
<command-3073683150227462> in <cell line: 47>()
     45 
     46 # Export to CSV
---> 47 df.to_csv('/mnt/data/oil_consumption_dataset.csv', index=False)

/databricks/python/lib/python3.9/site-packages/pandas/core/generic.py in to_csv(self, path_or_buf, sep, na_rep, float_format, columns, header, index, index_label, mode, encoding, compression, quoting, quotechar, line_terminator, chunksize, date_format, doublequote, escapechar, decimal, errors, storage_options)
   3464         )
   3465 
-> 3466         return DataFrameRenderer(formatter).to_csv(
   3467             path_or_buf,
   3468             line_terminator=line_terminator,

/databricks/python/lib/python3.9/site-packages/pandas/io/formats/format.py in to_csv(self, path_or_buf, encoding, sep, columns, index_label, mode, compression, quoting, quotechar, line_terminator, chunksize, 